In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/CLSSNE/

/content/drive/My Drive/Colab Notebooks/CLSSNE


In [4]:
!pip install rouge
!pip install sentencepiece
!pip install transformers

     |████████████████████████████████| 1.2 MB 9.6 MB/s 
     |████████████████████████████████| 3.4 MB 7.4 MB/s 
     |████████████████████████████████| 67 kB 7.2 MB/s 
     |████████████████████████████████| 596 kB 76.6 MB/s 
     |████████████████████████████████| 3.3 MB 54.7 MB/s 
     |████████████████████████████████| 895 kB 66.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 5.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=120999 sha256=c354743b2ae2173b95966038b2fd34148e23dac28327ca86d89895b65e8838f2
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import sys
import os
import argparse
import pandas as pd
import pickle
import torch
import torch.nn as nn
import logging
import torch.nn.functional as F # 激勵函數
import datetime
import ast
from transformers import MBartConfig, MBart50Tokenizer, MBartForConditionalGeneration, AdamW
from transformers import  get_linear_schedule_with_warmup
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset
from rouge import Rouge
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from gensim.summarization import keywords
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize

In [8]:
# Load Model
mBartPars_name = "model"
mBartPars_config = "{}/config.json".format(mBartPars_name)
mBartPars_tokenizer = "tokenizer"
mBartPars_model = "{}/pytorch_model.bin".format(mBartPars_name)
mBartConfig = MBartConfig.from_pretrained(mBartPars_config)
mBartTokenizer = MBart50Tokenizer.from_pretrained(mBartPars_tokenizer, src_lang="en_XX", tgt_lang="zh_CN")
mBartModel = MBartForConditionalGeneration.from_pretrained(mBartPars_model, config = mBartConfig)
print( 'Init : Model, Tokenizer success.' )

Init : Model, Tokenizer success.


In [9]:
# Load to GPU
mDevice = torch.device("cuda")
mBartModel.to(mDevice)
print('Init : Load model to Gpu success.')

Init : Load model to Gpu success.


In [10]:
def GetKeyword( article, nes ) :
    mSentences = sent_tokenize(article)
    mNameEntitys = ast.literal_eval(nes)
    finalSenteces = []
    for sen in mSentences :
        for ne in mNameEntitys :
            tmpRes = sen.find(ne)
            if tmpRes != -1 :
                finalSenteces.append(sen)
                break
    mergeText = ' '.join(finalSenteces)
    finalKeyword = keywords(mergeText).split('\n')
    return finalKeyword
def NameEntitysHandler(neListStr) :
    neHints_tmp = ast.literal_eval(neListStr)
    neHints = ','.join(neHints_tmp)
    return neHints

def NameEntitysHandlerWithSize(neHints_tmp) :
    if len(neHints_tmp) >= 3 : 
        neHints_tmp = neHints_tmp[:3]
    neHints = ','.join(neHints_tmp)
    return neHints

In [11]:
def PredictSummary( inputNameEntity, inputContext ):
    # tmpContext =  NameEntitysHandler(inputNameEntity) + '[CL]' + inputContext
    context_keyword = GetKeyword( inputContext, inputNameEntity )
    tmpContext =  NameEntitysHandler(inputNameEntity) + '[CL]' + NameEntitysHandlerWithSize(context_keyword) + '[CL]' + inputContext
    tmpInput = mBartTokenizer(tmpContext, max_length=522, pad_to_max_length=True, return_tensors='pt')
    tmpInput.to(mDevice)
    tmpOutput = mBartModel.generate( input_ids = tmpInput['input_ids'],
                                    attention_mask = tmpInput['attention_mask'],
                                    num_beams=4, max_length=120, repetition_penalty=1.2 ,early_stopping=True)
    tmpOutputStr = [ mBartTokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in tmpOutput ][0]
    print( inputNameEntity, " : ", tmpOutputStr)

In [12]:
# ==== COVID-19 ====

mArticle = '''
The Central Epidemic Command Center (CECC) yesterday announced a partial relaxation of the nation’s outdoor mask mandate as it reported zero new domestically transmitted COVID-19 cases.
Starting tomorrow, people would no longer need to wear masks in certain outdoor areas as long as social distancing can be maintained, said Minister of Health and Welfare Chen Shih-chung (陳時中), who heads the center.
Individuals in the agriculture, forestry, fishing and animal husbandry sectors who work in open spaces, such as fields and fish ponds, as well as people visiting forests and beaches, would not be required to wear masks, he said.
However, they would still have to carry a mask with them, as they would have to put it on if there is a sudden surge of people, he added.
Masks became a requirement for those using public transportation in April last year. A comprehensive outdoor mask mandate was introduced after Taiwan’s COVID-19 alert level was raised to level 3 in May, amid a spike in locally transmitted cases.
The alert level was lowered to level 2 on July 27, and several exceptions to mask requirements have since been announced, such as when having a meal, or when driving alone or with family members in an enclosed vehicle.
Further adjustments of mask requirements concerning specific occupations and religious events are to be announced today, Chen said.
Meanwhile, yesterday was the fourth consecutive day that zero new domestic cases have been reported, and the second straight day with no deaths, CECC data showed.
However, there were six new imported cases: two Taiwanese and four foreign nationals.
All six had recently arrived in Taiwan and tested positive during quarantine, the CECC said.
To date, Taiwan has confirmed a total of 16,250 COVID-19 cases, of which 14,417 were domestic infections reported after May 15, when the country first recorded more than 100 cases in a single day.
However, since Aug. 15, the daily number of domestic cases has mostly been in the single digits, totaling 114, CECC data showed.
With no new deaths reported yesterday, the number of confirmed COVID-19 deaths remained at 843, with all but 12 recorded since May 15, the data showed.
'''

PredictSummary("['Chen Shih-chung']",mArticle)
PredictSummary("['Taiwan','COVID-19','deaths']",mArticle)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


['Chen Shih-chung']  :  zh_CN 卫生部长陈志强说,人们不再需要在某些户外区域戴口罩。农业、林业、渔业和动物饲养部门的个人不需要戴口罩。然而,他们仍然必须带着一个面具,如果突然有人闯入,他们必须戴上。


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


['Taiwan','COVID-19','deaths']  :  zh_CN 昨天没有新的COVID-19病例报告,这是连续第四天。迄今为止,台湾已确认总共有16,250例COVID-19病例。自5月15日以来,已确认的COVID-19死亡人数仍为843人,自5月15日以来只有12人死亡。


In [16]:
mArticle = '''
A majority of Japanese feel friendly toward Taiwan, with almost half of respondents in a poll saying that they want to visit the country after COVID-19 travel curbs are eased, the Taipei Economic and Cultural Representative Office in Japan said yesterday.
The office said that 75.9 percent of respondents said they feel friendly toward Taiwan, citing as reasons the friendliness and politeness of Taiwanese, the long history of ties between the two nations, and the strength of bilateral trade.
More than one-quarter of respondents — 26.4 percent — said they had traveled to Taiwan, while 47.8 percent said they would like to once international travel resumes, the office said.
Regarding Taiwan-Japan relations, 71.4 percent of respondents said they are happy with the relationship, while 59.6 percent said they expect the ties to get a boost in the future, the office said.
The poll showed that 64.8 percent of respondents described Taiwan as “trustworthy,” citing as reasons Taiwan’s attitude toward Japan, shared democratic values and its peacefulness, it said.
Asked to choose the most friendly country in Asia, 46.6 percent of respondents chose Taiwan, 15.8 percent chose South Korea, 12.5 percent chose Singapore and 3 percent chose China, it said.
The poll showed that 65.1 percent of respondents agreed with Taipei’s assessment that trade between the two nations would get a boost if Taiwan joins the Comprehensive and Progressive Agreement for Trans-Pacific Partnership, it said.
Asked to choose which Taiwan-related issue is the most important to Japan, 40.7 percent said the country is most affected by the situation across the Taiwan Strait, while 8.2 percent cited fishery disputes and 6.8 cited economic competition, the office said.
Another 24.4 percent said there are no problems between Taiwan and Japan, it said.
Asked to select Taiwanese fruits they want to buy, 57.6 percent of respondents chose bananas, 57 percent chose pineapples and 44.2 percent chose mangoes, it said.
The office has since 2016 commissioned the Japan-based Central Research Services to conduct the annual survey on Japanese perception of Taiwan. This year’s poll utilized telephone interviews and Internet-based questionnaires, collecting responses from 1,000 Japanese aged 20 to 89.
'''

PredictSummary("['Taiwan']",mArticle)
PredictSummary("['Comprehensive']",mArticle)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


['Taiwan']  :  zh_CN 79%的受访者表示,他们对台湾感到友好。理由是友好和礼貌以及两国关系的悠久历史。超过四分之一的受访者——26.4%——表示他们已经去过台湾。


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


['Comprehensive']  :  zh_CN 79%的受访者表示,他们对台湾友好,理由是台湾人的友好和礼貌以及两国关系的悠久历史。超过四分之一的受访者——26.4%——表示他们已经去过台湾,而47.8 %的人表示,他们希望一旦国际旅行简历被提交,就可以访问台湾。调查显示,65.1%的受访者同意台北的评估,即如果台湾加入跨太平洋伙伴关系全面协议,两国间的贸易将得到推动。


In [22]:
mArticle = '''
A giant lantern shaped like a UFO that weighs more than 2 tonnes would be the main attraction at the Taipei Lantern Festival, which is to start in Taipei’s Shilin District (士林) on Feb. 11, the Taipei City Government said yesterday.
The UFO, which measures 7m in diameter and stands at 5m tall, lights up with spotlights and an array of LEDs, including panels that display the festival’s mascot, “Auspicious Tiger,” in the cockpit, Taipei Deputy Mayor Tsai Ping-kun (蔡炳坤) told a news conference.
The tiger is next lunar year’s zodiac animal, traditionally symbolizing vigor and vitality.
Festival creative director Akibo (李明道) said that the idea of the 2.5-tonne UFO was developed from an animation concept of a cosmic UFO that became lost in space and landed in Shilin.
Taipei then invited the travelers on the UFO to display the spacecraft as the festival’s main lantern, Akibo said.
“We hope it will spark the imagination of visitors and also give them an opportunity to broaden their horizons to give them unlimited possibilities,” Akibo said.
During the festival, the UFO would be the centerpiece of a three-minute show every 30 minutes in which it would be lifted 6m into the air to make it look like as if it is flying, Akibo said.
The UFO is one of Akibo’s many robotic-designed art installations across Taipei, including a Bluetooth speaker robot at Xinzhongshan Linear Park (心中山線形公園) and a treasure-finding airship at Taipei Children’s Amusement Park, Akibo said.
“When I was young, I didn’t have toys to play with, so when I grew up, I wanted to make a lot of big toys for myself and my children,” Akibo said.
The 10-day festival, which is to run through Feb. 20, would cover areas along three stations of the MRT railway serving metropolitan Taipei — including MRT Jiantan Station, which serves Shilin Night Market (士林夜市) — and showcase more than 50 lanterns and lantern artworks, the Taipei Department of Information and Tourism said in a statement.
The festival would also have two stage areas to host shows and performances, including by Golden Melody Award-winning artist Sanpuy Katatepan Mavaliyw and AKB48 Team TP, it added.

'''

PredictSummary("['UFO','Taipei']",mArticle)
PredictSummary("['Akibo','Xinzhongshan ']",mArticle)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


['UFO','Taipei']  :  zh_CN 一个形状像不明飞行物的巨大灯笼将是台北灯笼节的主要吸引力。这种2.5吨重的不明飞行物直径为7米,高5米,用聚光灯和一系列发光二极管点亮。开幕式将于2月11日开始,为期10天的活动将展出50多件灯笼和灯笼艺术品。


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


['Akibo','Xinzhongshan ']  :  zh_CN 一个形状像不明飞行物的巨大灯笼将是台北灯笼节的主要吸引力。这只2.5吨重的不明飞行物直径为7米,长度为5米,用聚光灯和一系列发光二极管点亮。创意总监阿基博说,这个想法是基于一个在太空迷失的宇宙不明飞行物的动画概念。不明飞行物是阿基博在台北的许多机器人设计的艺术装置之一,包括兴盛山线性公园的一个蓝牙扬声器机器人。
